In [ ]:
!pip install langchain
!pip install langchain_core
!pip install langchain_openai
!pip install langchain_community

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [2]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql",
    filename="Chinook_Sqlite.sql",
)

('Chinook_Sqlite.sql', <http.client.HTTPMessage at 0x1e0398d1850>)

In [3]:
import sqlite3

conn = sqlite3.connect('Chinook.db')

with open("Chinook_Sqlite.sql", 'r', encoding='UTF-8') as file:
    script = file.read()

conn.executescript(script)
conn.close()

In [4]:
# from langchain_community.utilities import SQLDatabase
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [5]:
db.run("select * from Artist limit 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [6]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI



In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [23]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# LangChain이 지원하는 다른 채팅 모델을 사용합니다. 여기서는 Ollama를 사용합니다.
llm = ChatOllama(model="EEVE-Korean-10.8B:latest")

In [29]:
from langserve import RemoteRunnable

# llm = RemoteRunnable("http://192.168.0.12:8000/prompt/")
llm = RemoteRunnable("http://192.168.0.12:8000/llm/")

In [30]:
chain = create_sql_query_chain(llm, db)

In [ ]:
# response = chain.invoke({"question": "empoloyee는 몇명이 있어?"})
response = chain.invoke({"question": "empoloyee는 몇명이 있어?"})

In [26]:
response

'SELECT COUNT(*) FROM Employee;'

In [27]:
db.run(response)

'[(8,)]'

In [28]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "empoloyee는 몇명이 있어?"})

'[(8,)]'

In [7]:
prompt_template = ChatPromptTemplate.from_template("""Based on the table schema below, write a SQL query that would answer the user's question:{schema} Question: {question} SQL Query:""")

In [8]:
def get_schema(_):
    return db.get_table_info()

In [10]:
llm

In [14]:
chain = (RunnablePassthrough.assign(schema = get_schema)
         | prompt_template
        #  | ChatOpenAI()
         | llm
         | StrOutputParser()
        #  | db.run
         )

In [15]:
db.get_table_info()

'\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEG

In [16]:
prompt_template

ChatPromptTemplate(input_variables=['question', 'schema'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'schema'], template="Based on the table schema below, write a SQL query that would answer the user's question:{schema} Question: {question} SQL Query:"))])

In [63]:
chain.invoke({"question":'점원의 수는?'})

"\n'None'이란 특정 객체나 항목이 존재하지 않거나 해당되는 것이 없을 때 사용되며, 무언가나 누군가가 부재함을 나타내는 단어입니다. '없음', '아무것도 아님', 또는 '무관한 것'을 의미하기도 합니다. 문법적으로 'none'은 명사이며 보통 부정 관사와 함께 쓰이거나 복수 명사의 대명사로 사용됩니다:\n\n- 부정관사와 함께 (단수 사용): There is none left.\n- 복수 명사의 대명사로서 (복수 사용): None of the fruits are ripe.\n\n간단히 말해서, 'none'은 무언가가 존재하지 않거나 관련성이 없는 것을 설명하는 데 도움이 되는 단어입니다."

In [ ]:
chain.invoke({"question":'가장 매출이 적은 점원은?'})

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 데이터베이스 연결 설정
# DATABASE_TYPE = 'mysql'
# DBAPI = 'pymysql'
# USER = 'your_username'
# PASSWORD = 'your_password'
# HOST = 'your_host'
# PORT = 'your_port'
# DATABASE = 'your_database'

# SQLAlchemy 엔진 생성
# engine = create_engine(f'{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

# 로컬 LLM 모델 설정
model_name = "t5-small"  # 또는 적합한 다른 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_sql_query(natural_language_query):
    inputs = tokenizer.encode("translate English to SQL: " + natural_language_query, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, num_beams=5, early_stopping=True)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query

# 예시 자연어 쿼리
natural_language_query = "Show me all records from the customers table where the customer lives in New York."

# SQL 쿼리 생성
sql_query = generate_sql_query(natural_language_query)
print(f"Generated SQL Query: {sql_query}")

# SQL 쿼리 실행
result = pd.read_sql(sql_query, engine)

# 결과 출력
print(result.head())


In [22]:
from langserve import RemoteRunnabl

ImportError: cannot import name 'RemoteRunnabl' from 'langserve' (c:\dev2\teddylee777\langchain-kr\.venv\Lib\site-packages\langserve\__init__.py)

In [26]:
import requests
import json
import pandas as pd
from sqlalchemy import create_engine

# Ollama API 로컬 서버 URL
# ollama_url = "http://localhost:5000/generate"

# ollama_url = ChatOllama(model="EEVE-Korean-10.8B:latest")

from langserve import RemoteRunnable
# ollama_url = RemoteRunnable("http://192.168.0.12:8000/prompt/")
ollama_url = RemoteRunnable("http://192.168.0.12:8000/generate/")

# Chinook 데이터베이스 연결 설정
DATABASE = 'sqlite:///Chinook.sqlite'  # SQLite 데이터베이스 파일 경로
engine = create_engine(DATABASE)


In [30]:
ollama_url

In [27]:

def generate_sql_query(natural_language_query):
    payload = {
        "prompt": f"Convert the following natural language query to SQL: {natural_language_query}",
        "max_tokens": 100
    }
    headers = {'Content-Type': 'application/json'}
    
    response = requests.post(ollama_url, data=json.dumps(payload), headers=headers)
    sql_query = response.json()['choices'][0]['text'].strip()
    return sql_query

# 예시 자연어 쿼리
natural_language_query = "Show me all tracks by AC/DC."

# SQL 쿼리 생성
sql_query = generate_sql_query(natural_language_query)
print(f"Generated SQL Query: {sql_query}")

# SQL 쿼리 실행
result = pd.read_sql(sql_query, engine)

# 결과 출력
print(result.head())


MissingSchema: Invalid URL '<langserve.client.RemoteRunnable object at 0x000001E04D7FCD50>': No scheme supplied. Perhaps you meant https://<langserve.client.RemoteRunnable object at 0x000001E04D7FCD50>?

In [32]:
import pandas as pd
import sqlite3
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 데이터베이스 연결 설정
DATABASE = 'Chinook.sqlite'  # SQLite 데이터베이스 파일 경로
conn = sqlite3.connect(DATABASE)

# 로컬 모델 설정
model_name = "t5-small"  # 경량화된 T5 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_sql_query(natural_language_query):
    inputs = tokenizer.encode("translate English to SQL: " + natural_language_query, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, num_beams=5, early_stopping=True)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query

def execute_sql_query(sql_query, connection):
    # SQL 쿼리 실행
    result = pd.read_sql(sql_query, connection)
    return result

# 예시 자연어 쿼리
natural_language_query = "Show me all tracks by AC/DC."

# SQL 쿼리 생성
sql_query = generate_sql_query(natural_language_query)
print(f"Generated SQL Query: {sql_query}")

# SQL 쿼리 실행 및 결과 출력
result = execute_sql_query(sql_query, conn)
print(result.head())


Generated SQL Query: Show me all tracks by AC/DC.


DatabaseError: Execution failed on sql 'Show me all tracks by AC/DC.': near "Show": syntax error